# Imersão Aluna

## Bibliotecas

In [1]:
file = "/content/Imersao.xlsx"

import pandas as pd
import numpy as np
import plotly.express as px

## Dados

In [2]:
df_principal = pd.read_excel(file, sheet_name="Principal")

df_total_acoes = pd.read_excel(file, sheet_name="Total_de_acoes")

df_ticker = pd.read_excel(file, sheet_name="Ticker")

df_origem = pd.read_excel(file, sheet_name="Origem")

pd.options.display.float_format = "{:.2f}".format

df_principal

,Ativo,Data,Último (R$),Var. Dia (%),Var. Sem. (%),Var. Mês (%),Var. Ano (%),Var. 12M (%),Val. Mín,Val. Máx,Volume,Variação,Valor inicial(R$),Quantidade de ações,Variação R$,Resultado,Nome da empresa,Segmento,Idade,Cat idade
0,USIM5,2024-01-26,9.50,5.20,11.76,2.26,2.26,15.97,9.18,9.56,"319,16 M",0.05,9.03,515117391,241889725.40,Subiu👍,Usiminas,Siderurgia,68,Entre 50 e 100 anos
1,CMIN3,2024-01-26,6.82,2.40,2.40,-12.11,-12.11,50.56,6.66,6.86,"32,65 M",0.02,6.66,1110559345,177515970.30,Subiu👍,CSN Mineração,Mineração,82,Entre 50 e 100 anos
2,PETR3,2024-01-26,41.96,2.19,7.73,7.64,7.64,77.55,40.81,42.34,"436,69 M",0.02,41.06,2379877655,2140059394.00,Subiu👍,Petrobras,Petróleo e Gás,71,Entre 50 e 100 anos
3,SUZB3,2024-01-26,52.91,2.04,2.14,-4.89,-4.89,18.85,51.89,53.17,"162,55 M",0.02,51.85,683452836,722946282.70,Subiu👍,Suzano,Papel e Celulose,99,Entre 50 e 100 anos
4,CPFE3,2024-01-26,37.10,2.03,2.49,-3.66,-3.66,20.70,36.37,37.32,"73,42 M",0.02,36.36,187732538,138573955.10,Subiu👍,CPFL Energia,Energia Elétrica,123,Mais de 100 anos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,CRFB3,2024-01-26,10.71,-2.45,-9.47,-13.98,-13.98,-32.72,10.70,11.08,"38,19 M",-0.02,10.98,533990587,-143635530.60,Desceu👎,Carrefour Brasil,Varejo alimentar,74,Entre 50 e 100 anos
77,BHIA3,2024-01-26,8.70,-2.46,-6.95,-23.55,-23.55,-85.74,8.67,8.95,"27,54 M",-0.02,8.92,94843047,-20810240.84,Desceu👎,Casas Bahia,Varejo de eletrodomésticos e móveis,71,Entre 50 e 100 anos
78,RENT3,2024-01-26,56.24,-3.63,-6.41,-11.57,-11.57,-2.77,56.04,58.90,"624,74 M",-0.04,58.36,853202347,-1807432634.00,Desceu👎,Localiza,Aluguel de carros,50,Entre 50 e 100 anos
79,CVCB3,2024-01-26,3.07,-4.36,-5.54,-12.29,-12.29,-36.83,3.05,3.23,"101,46 M",-0.04,3.21,525582771,-73557408.06,Desceu👎,CVC,Turismo,50,Entre 50 e 100 anos


## Modelagem de dados

In [3]:
# Renomear os parametros
df_principal = df_principal.rename(columns={"Último (R$)":"valor_final", "Var. Dia (%)":"var_dia_pct", "Valor inicial(R$)":"valor_inicial"}).copy()

## Unir as colunas de total de acoes
df_principal = df_principal.merge(df_total_acoes, left_on="Ativo", right_on="Código", how="left").copy()
df_principal= df_principal.drop(columns=['Código'])

## Unir as colunas de Ticker
df_principal = df_principal.merge(df_ticker, left_on="Ativo", right_on="Ticker", how="left").copy()
df_principal= df_principal.drop(columns=['Ticker'])

## Unir as colunas de Origem
df_origem= df_origem.drop(columns=['Segmento'])
df_principal = df_principal.merge(df_origem, left_on="Nome", right_on="Nome", how="left")
df_principal = df_principal.rename(columns={"Idade (em anos)":"idade"}).copy()

# Transformar a colunas em inteiro, pois estava formatada com float
df_principal['Qtde. Teórica'] = df_principal['Qtde. Teórica'].astype(int)
df_principal = df_principal.rename(columns={"Qtde. Teórica":"qtd_teorica"}).copy()

## Operacoes

In [4]:
# Criar a coluna de variacao em RS
df_principal['variacao_rs'] = (df_principal['valor_final'] - df_principal['valor_inicial']) * df_principal['qtd_teorica']

# Criar a coluna resultado, para verificar o status da variacao em reais(positiva, neutra ou negativa)
df_principal['resultado'] = np.where(df_principal['variacao_rs'] > 0, 'Subiu👍', np.where(df_principal['variacao_rs'] == 0, 'Estável😇', 'Desceu👎'))

## Criar a coluna categoria de idades, para criar um intervalo entre das idades
df_principal['cat_idade'] = np.where(df_principal['idade'] > 100, 'Mais de 100 anos', np.where(df_principal['idade'] < 50, 'Menos de 50 anos', 'Entre 50 e 100 anos'))


In [5]:
df_principal.head(5)


,Ativo,Data,valor_final,var_dia_pct,Var. Sem. (%),Var. Mês (%),Var. Ano (%),Var. 12M (%),Val. Mín,Val. Máx,...,Nome da empresa,Segmento,Idade,Cat idade,qtd_teorica,Nome,idade,variacao_rs,resultado,cat_idade
0,USIM5,2024-01-26,9.50,5.20,11.76,2.26,2.26,15.97,9.18,9.56,...,Usiminas,Siderurgia,68,Entre 50 e 100 anos,515117391,Usiminas,68,241889725.41,Subiu👍,Entre 50 e 100 anos
1,CMIN3,2024-01-26,6.82,2.40,2.40,-12.11,-12.11,50.56,6.66,6.86,...,CSN Mineração,Mineração,82,Entre 50 e 100 anos,1110559345,CSN Mineração,82,177515970.30,Subiu👍,Entre 50 e 100 anos
2,PETR3,2024-01-26,41.96,2.19,7.73,7.64,7.64,77.55,40.81,42.34,...,Petrobras,Petróleo e Gás,71,Entre 50 e 100 anos,2379877655,Petrobras,71,2140059382.80,Subiu👍,Entre 50 e 100 anos
3,SUZB3,2024-01-26,52.91,2.04,2.14,-4.89,-4.89,18.85,51.89,53.17,...,Suzano,Papel e Celulose,99,Entre 50 e 100 anos,683452836,Suzano,99,722946281.15,Subiu👍,Entre 50 e 100 anos
4,CPFE3,2024-01-26,37.10,2.03,2.49,-3.66,-3.66,20.70,36.37,37.32,...,CPFL Energia,Energia Elétrica,123,Mais de 100 anos,187732538,CPFL Energia,123,138573954.41,Subiu👍,Mais de 100 anos


In [6]:
maior_valor = df_principal['variacao_rs'].max()
menor_valor = df_principal['variacao_rs'].min()

media_geral = df_principal['variacao_rs'].mean()

media_subiu = df_principal[df_principal['resultado'] == 'Subiu👍']['variacao_rs'].mean()
media_desceu = df_principal[df_principal['resultado'] == 'Desceu👎']['variacao_rs'].mean()

print(f"Maior valor: {maior_valor}")
print(f"Menor valor: {menor_valor}")
print(f"Média geral: {media_geral}")
print(f"Média dos valores que subiram: {media_subiu}")
print(f"Média dos valores que desceram: {media_desceu}")

Maior valor: 4762926985.257359
Menor valor: -1807432631.3369505
Média geral: 165190210.37501976
Média dos valores que subiram: 448164249.30019546
Média dos valores que desceram: -181109140.8237715


## Graficos

In [7]:
## Pegar todos os resultados que subiram
df_principal_subiu= df_principal[df_principal['resultado'] == 'Subiu👍']
# Agrupar por segmento e calcular a soma entre os segmentos
df_analise_segmento = df_principal.groupby('Segmento')['variacao_rs'].sum().reset_index()


# Crie o gráfico de pizza
fig = px.pie(df_analise_segmento, values="variacao_rs", names="Segmento")

# Opcional: personalize o gráfico
fig.update_layout(title="Variacao RS por resultado")

# Exiba o gráfico
fig.show()

In [8]:
# Agrupar por resultando e calcular a soma entre variacaos
df_principal_saldo = df_principal.groupby('resultado')['variacao_rs'].sum().reset_index()
# Criar um grafico em barras
fig = px.bar(df_principal_saldo, x="resultado", y="variacao_rs", text="variacao_rs", title="Variacao RS por resultado")
fig.show()